In [1]:
import fs from "node:fs";
import { dirname } from "node:path";
import { exec } from "node:child_process";

import * as Plot from "npm:@observablehq/plot";
import * as d3 from "npm:d3";
import { JSDOM } from "npm:jsdom";
import * as pl from "npm:nodejs-polars";
import { readCSV } from "npm:nodejs-polars";
import puppeteer from "npm:puppeteer";


In [2]:
function uniformSample<D extends pl.DataFrame>(df: D, targetSize: number = 1000): D {
  if (df.height <= targetSize) return df;

  const sampledColumns = df.columns.map(() => []);
  const ratio = df.height / targetSize;

  for (let i = 0; i < targetSize; i++) {
    const index = Math.round(i * ratio);
    const row = df.row(index);
    for (let j = 0; j < df.width; j++) sampledColumns[j].push(row[j]);
  }

  return pl.DataFrame(Object.fromEntries(sampledColumns.map((col, i) => [df.columns[i], col])));
}


In [3]:
async function savePNG(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
  } = {}
) {
  const { padding = 0 } = options;

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  await page.setViewport({
    // Simulate 4K resolution
    width: 3840,
    height: 2160,
    deviceScaleFactor: 2, // High DPI for better quality
  });

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  // Capture the figure element
  const figureElement = (await page.$("figure")) || (await page.$("svg"));

  if (!figureElement) throw new Error("Figure element not found");

  const boundingBox = await figureElement.boundingBox();

  if (!boundingBox) throw new Error("Could not get figure bounding box");

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  // Screenshot
  await figureElement.screenshot({
    path,
    type: "png",
    omitBackground: false, // Include background
  });

  await browser.close();
}

async function savePDF(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
    /** Paper format */
    format?: "A4" | "A3" | "A5" | "Letter" | "Legal" | "Tabloid";
    printBackground?: boolean;
  } = {}
) {
  const { padding = 0, format = "A4", printBackground = true } = options;

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; font-family: 'Times New Roman', serif; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "      figure { margin: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  await page.pdf({
    path,
    format,
    printBackground,
    preferCSSPageSize: true,
  });

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  await browser.close();
}

async function saveEPS(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
    printBackground?: boolean;
  } = {}
) {
  const { padding = 0, printBackground = true } = options;

  const tempFilePath = path.replace(/\.eps$/, "") + "-" + Date.now() + ".pdf";

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; font-family: 'Times New Roman', serif; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "      figure { margin: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  const figure = (await page.$("figure")) || (await page.$("svg"));
  if (!figure) throw new Error("Figure element not found");
  const box = await figure.boundingBox();
  if (!box) throw new Error("Could not compute bounding box");

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  await page.pdf({
    path: tempFilePath,
    printBackground,
    width: `${Math.ceil(box.width)}px`,
    height: `${Math.ceil(box.height)}px`,
  });

  await browser.close();

  try {
    await new Promise((resolve, reject) => {
      exec(`inkscape "${tempFilePath}" --export-filename="${path}"`, (error) => {
        if (error) reject(error);
        else resolve(null);
      });
    });
  } catch (error) {
    console.error("inkscape not found. Please install Inkscape");
    console.error("Ubuntu/Debian: sudo apt install inkscape");
    console.error("macOS: brew install inkscape");
  }

  await fs.promises.rm(tempFilePath, { force: true });
}


In [ ]:
const df = readCSV(fs.readFileSync("meta_large_miss_ratio.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("W-TinyLFU_CMS").as("CMS"),
  pl.col("W-TinyLFU_ADA").as("Ada"),
  pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evolving (Pruning Only)"),
  pl.col("W-TinyLFU_EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.089, 0.154],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_meta_large.eps");
Plot.plot({
  ...options,
  title: "Meta KV (Large) - Miss Ratio V.S. Alpha",
});


In [ ]:
const df = readCSV(fs.readFileSync("meta_small_miss_ratio.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("W-TinyLFU_CMS").as("CMS"),
  pl.col("W-TinyLFU_ADA").as("Ada"),
  pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evolving (Pruning Only)"),
  pl.col("W-TinyLFU_EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.158, 0.275],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_meta_small.eps");
Plot.plot({
  ...options,
  title: "Meta KV Trace (Small) - Miss Ratio V.S. Alpha",
});


Meta KV Trace (Small) - Miss Ratio V.S. Alpha Ada CMS Evolving Evolving (Pruning Only) 0.16 0.18 0.20 0.22 0.24 0.26 Miss Ratio 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [ ]:
const df = readCSV(fs.readFileSync("msr_large_miss_ratio.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("W-TinyLFU_CMS").as("CMS"),
  pl.col("W-TinyLFU_ADA").as("Ada"),
  pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evolving (Pruning Only)"),
  pl.col("W-TinyLFU_EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.0015, 0.0315],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 108,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_msr_large.eps");
Plot.plot({
  ...options,
  title: "MSR Trace (Large) - Miss Ratio V.S. Alpha",
});


MSR Trace (Large) - Miss Ratio V.S. Alpha Ada CMS Evolving Evolving (Pruning Only) 0.005 0.010 0.015 0.020 0.025 0.030 Miss Ratio 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [ ]:
const df = readCSV(fs.readFileSync("msr_small_miss_ratio.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("W-TinyLFU_CMS").as("CMS"),
  pl.col("W-TinyLFU_ADA").as("Ada"),
  pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evolving (Pruning Only)"),
  pl.col("W-TinyLFU_EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.405, 0.525],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_msr_small.eps");
Plot.plot({
  ...options,
  title: "MSR Trace (Small) - Miss Ratio V.S. Alpha",
});


MSR Trace (Small) - Miss Ratio V.S. Alpha Ada CMS Evolving Evolving (Pruning Only) 0.42 0.44 0.46 0.48 0.50 0.52 Miss Ratio 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [ ]:
const df = readCSV(fs.readFileSync("hm_large_coverage.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("CMS"),
  pl.col("ADA").as("Ada"),
  pl.col("EVO_PRUNING_ONLY").as("Evolving (Pruning Only)"),
  pl.col("EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, trendingCoverage]) => ({ alpha: record.alpha, baseline, trendingCoverage }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "Trending Coverage",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.22, 0.67],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "trendingCoverage",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "trendingCoverage",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 84,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/coverage_hm_large.eps");
Plot.plot({
  ...options,
  title: "H&M Trending (K=5000) - Coverage V.S. Alpha",
});


H&M Trending (K=5000) - Coverage V.S. Alpha Ada CMS Evolving Evolving (Pruning Only) 0.3 0.4 0.5 0.6 Trending Coverage 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 2,000 5,000 10,000 (Initial) Alpha

In [ ]:
const df = readCSV(fs.readFileSync("hm_small_coverage.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("CMS"),
  pl.col("ADA").as("Ada"),
  pl.col("EVO_PRUNING_ONLY").as("Evolving (Pruning Only)"),
  pl.col("EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, trendingCoverage]) => ({ alpha: record.alpha, baseline, trendingCoverage }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "Trending Coverage",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.03, 0.27],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "trendingCoverage",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "trendingCoverage",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/coverage_hm_small.eps");
Plot.plot({
  ...options,
  title: "H&M Trending (K=1000) - Coverage V.S. Alpha",
});


H&M Trending (K=1000) - Coverage V.S. Alpha Ada CMS Evolving Evolving (Pruning Only) 0.05 0.10 0.15 0.20 0.25 Trending Coverage 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 2,000 5,000 10,000 (Initial) Alpha

In [ ]:
const queryLatency = readCSV(fs.readFileSync("msr_small_query_throughput.csv", "utf-8"))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evolving-PO"),
    pl.col("W-TinyLFU_EVO").as("Evolving")
  )
  .filter(pl.col("alpha").eq(1))
  .drop("alpha")
  .toRecords()[0];
const updateLatency = readCSV(fs.readFileSync("msr_small_update_throughput.csv", "utf-8"))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_PRUNING_ONLY").as("Evolving-PO"),
    pl.col("W-TinyLFU_EVO").as("Evolving")
  )
  .filter(pl.col("alpha").eq(1))
  .drop("alpha")
  .toRecords()[0];
const data = Object.keys(queryLatency).flatMap((baseline) => [
  { baseline, type: "Query", throughput: 1.0 / queryLatency[baseline] / 1_000_000 },
  { baseline, type: "Update", throughput: 1.0 / updateLatency[baseline] / 1_000_000 },
]);
const options = {
  color: { legend: true },
  x: { label: null },
  y: {
    label: "Throughput (MOps)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0, 23],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.barY(data, {
      x: "baseline",
      fill: "type",
      opacity: 0, // Only used to add legend
    }),
    Plot.barY(
      data.filter((d) => d.type === "Query"),
      {
        x: "baseline",
        y: "throughput",
        insetRight: 62,
        fill: "#4269d0",
        stroke: "black",
        strokeWidth: 3,
      }
    ),
    Plot.barY(
      data.filter((d) => d.type === "Update"),
      {
        x: "baseline",
        y: "throughput",
        insetLeft: 62,
        fill: "#f0b018",
        stroke: "black",
        strokeWidth: 3,
      }
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 76,
  style: { fontSize: "24px" },
  x: { ...options.x },
});
d3.select(plot)
  .select("div")
  .style("font-size", "24px")
  .style("margin-left", "220px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/throughput_msr.eps");
Plot.plot({
  ...options,
  title: "Query & Update Throughput on MSR",
});


Query & Update Throughput on MSR Query Update 0 5 10 15 20 Throughput (MOps) Ada CMS Evolving Evolving-PO

In [ ]:
const queryLatency = readCSV(fs.readFileSync("hm_small_query_throughput.csv", "utf-8"))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_PRUNING_ONLY").as("Evolving-PO"),
    pl.col("EVO").as("Evolving")
  )
  .filter(pl.col("alpha").eq(1))
  .drop("alpha")
  .toRecords()[0];
const updateLatency = readCSV(fs.readFileSync("hm_small_update_throughput.csv", "utf-8"))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_PRUNING_ONLY").as("Evolving-PO"),
    pl.col("EVO").as("Evolving")
  )
  .filter(pl.col("alpha").eq(1))
  .drop("alpha")
  .toRecords()[0];
const data = Object.keys(queryLatency).flatMap((baseline) => [
  { baseline, type: "Query", throughput: 1.0 / queryLatency[baseline] / 1_000_000 },
  { baseline, type: "Update", throughput: 1.0 / updateLatency[baseline] / 1_000_000 },
]);
const options = {
  color: { legend: true },
  x: { label: null },
  y: {
    label: "Throughput (MOps)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0, 23.9],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.barY(data, {
      x: "baseline",
      fill: "type",
      opacity: 0, // Only used to add legend
    }),
    Plot.barY(
      data.filter((d) => d.type === "Query"),
      {
        x: "baseline",
        y: "throughput",
        insetRight: 62,
        fill: "#4269d0",
        stroke: "black",
        strokeWidth: 3,
      }
    ),
    Plot.barY(
      data.filter((d) => d.type === "Update"),
      {
        x: "baseline",
        y: "throughput",
        insetLeft: 62,
        fill: "#f0b018",
        stroke: "black",
        strokeWidth: 3,
      }
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 76,
  style: { fontSize: "24px" },
  x: { ...options.x },
});
d3.select(plot)
  .select("div")
  .style("font-size", "24px")
  .style("margin-left", "220px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/throughput_hm.eps");
Plot.plot({
  ...options,
  title: "Query & Update Throughput on H&M",
});


Query & Update Throughput on H&M Query Update 0 5 10 15 20 Throughput (MOps) Ada CMS Evolving Evolving-PO

In [6]:
const df = readCSV(fs.readFileSync("../output/msr.alpha_1.trace.csv", "utf-8")).select(
  pl.col("Objective").as("hitRate"),
  pl.col("Parameter").as("alpha")
);
console.log("Trace length:", df.height);
console.log(df.describe().toString());
const data = uniformSample(df, 1000)
  .toRecords()
  .map((d, i) => ({ index: i, ...d }));
const y2 = d3.scaleLog(
  d3.extent(data, (d) => d.alpha),
  d3.extent(data, (d) => 1 - d.hitRate)
);
const options = {
  x: { label: "Time", labelAnchor: "center" },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.axisY({
      axis: "left",
      color: "hsl(207, 45%, 30%)",
      label: "Miss Ratio",
      labelAnchor: "center",
      labelArrow: false,
      ticks: 5,
    }),
    Plot.axisY(y2.ticks(), {
      anchor: "right",
      color: "hsl(0, 53%, 40%)",
      label: "Alpha",
      labelAnchor: "center",
      labelArrow: false,
      y: y2,
      tickFormat: (d) => {
        let s = y2.tickFormat()(d);
        if (s === "200m") s = "0.2";
        return s;
      },
    }),
    Plot.lineY(data, { x: "index", y: (d) => 1 - d.hitRate, stroke: "steelblue" }),
    Plot.dotY(
      data,
      Plot.mapY((D) => D.map(y2), { x: "index", y: (d) => d.alpha, r: 1, stroke: "indianred" })
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 64,
  marginRight: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 56 },
});
d3.select(plot).select("div").style("font-size", "20px");
await saveEPS(plot, "../output/figures/adaptation_timeline_msr.eps");
Plot.plot({
  ...options,
  title: "MSR Trace - Adaptation Timeline",
});


Trace length: 1686
shape: (5, 3)
┌──────────┬──────────┬────────────┐
│ describe ┆ hitRate  ┆ alpha      │
│ ---      ┆ ---      ┆ ---        │
│ str      ┆ f64      ┆ f64        │
╞══════════╪══════════╪════════════╡
│ mean     ┆ 0.997508 ┆ 59.620677  │
│ std      ┆ 0.007473 ┆ 132.426322 │
│ min      ┆ 0.85672  ┆ 0.1        │
│ max      ┆ 0.99996  ┆ 1000.0     │
│ median   ┆ 0.99943  ┆ 13.848864  │
└──────────┴──────────┴────────────┘


MSR Trace - Adaptation Timeline 0 200 400 600 800 Time → 0.02 0.04 0.06 0.08 0.10 0.12 0.14 Miss Ratio 0.2 1 2 10 20 100 200 Alpha <path d="M40,20L40.561,338.04L41.121,339.506L41.682,340.654L42.242,339.628L42.803,339.384L43.363,339.115L43.924,343.171L44.484,341.68L45.045,342.267L45.606,366.457L46.166,366.799L46.727,367.434L47.287,348.986L47.848,367.825L48.408,365.748L48.969,366.53L49.53,364.185L50.09,364.893L50.651,366.286L51.211,368.754L51.772,366.921L52.332,365.651L52.893,364.576L53.453,367.141L54.014,364.429L54.575,368.216L55.135,363.843L55.696,367.557L56.256,366.555L56.817,369.218L57.377,369.462L57.938,369.34L58.498,369.047L59.059,369.291L59.62,369.267L60.18,368.949L60.741,369.218L61.301,369.12L61.862,368.485L62.422,368.852L62.983,363.843L63.544,368.925L64.104,369.096L64.665,366.652L65.225,368.607L65.786,368.119L66.346,367.63L66.907,368.216L67.467,368.167L68.028,368.754L68.589,368.681L69.149,368.852L69.71,368.583L70.27,366.701L70.831,369.194L71.391,368.9L71.952,368.387L72.513,368.974L73.073,368.9L73.634,368.632L74.194,368.729L74.755,368.485L75.315,368.998L75.876,369.316L76.436,368.803L76.997,369.096L77.558,369.145L78.118,368.974L78.679,369.071L79.239,369.316L79.8,369.12L80.36,369.243L80.921,369.389L81.481,369.291L82.042,369.365L82.603,369.438L83.163,369.243L83.724,368.192L84.284,367.948L84.845,369.34L85.405,369.243L85.966,369.365L86.527,368.778L87.087,368.998L87.648,369.365L88.208,364.893L88.769,362.914L89.329,368.558L89.89,369.267L90.45,366.75L91.011,368.485L91.572,368.461L92.132,368.754L92.693,368.29L93.253,367.703L93.814,363.549L94.374,365.064L94.935,359.273L95.495,361.57L96.056,364.82L96.617,360.202L97.177,366.164L97.738,356.805L98.298,362.425L98.859,361.326L99.419,363.183L99.98,364.111L100.541,364.6L101.101,354.02L101.662,363.085L102.222,364.649L102.783,356.121L103.343,352.236L103.904,357.661L104.464,365.235L105.025,364.16L105.586,366.139L106.146,362.279L106.707,362.963L107.267,354.435L107.828,360.568L108.388,364.918L108.949,345.443L109.51,357.392L110.07,353.409L110.631,360.275L111.191,366.75L111.752,365.651L112.312,358.467L112.873,364.698L113.433,123.089L113.994,365.626L114.555,364.404L115.115,359.127L115.676,366.188L116.236,357.318L116.797,365.162L117.357,363.452L117.918,352.676L118.478,361.595L119.039,367.361L119.6,366.237L120.16,364.062L120.721,363.549L121.281,367.532L121.842,361.766L122.402,364.014L122.963,366.677L123.524,363.745L124.084,366.555L124.645,361.155L125.205,366.579L125.766,364.942L126.326,367.728L126.887,369.267L127.447,369.414L128.008,369.853L128.569,369.145L129.129,367.948L129.69,369.682L130.25,369.487L130.811,369.756L131.371,369.707L131.932,369.902L132.492,369.633L133.053,369.56L133.614,369.927L134.174,369.658L134.735,369.829L135.295,369.951L135.856,369.682L136.416,369.805L136.977,369.902L137.538,369.902L138.098,369.853L138.659,368.607L139.219,365.064L139.78,369.609L140.34,369.805L140.901,369.682L141.461,369.902L142.022,369.756L142.583,369.658L143.143,369.756L143.704,369.853L144.264,369.707L144.825,369.927L145.385,365.993L145.946,368.729L146.507,369.096L147.067,368.363L147.628,368.632L148.188,368.387L148.749,368.192L149.309,368.852L149.87,368.754L150.43,368.998L150.991,368.363L151.552,368.632L152.112,367.361L152.673,368.925L153.233,368.729L153.794,368.803L154.354,368.705L154.915,369.096L155.475,369.218L156.036,368.998L156.597,369.071L157.157,369.071L157.718,369.194L158.278,368.925L158.839,368.632L159.399,368.729L159.96,369.316L160.521,368.632L161.081,369.267L161.642,369.047L162.202,369.414L162.763,369.023L163.323,369.218L163.884,369.389L164.444,369.316L165.005,369.316L165.566,368.681L166.126,369.243L166.687,369.34L167.247,369.365L167.808,369.243L168.368,369.56L168.929,369.267L169.489,369.316L170.05,369.389L170.611,369.462L171.171,369.365L171.732,369.609L172.292,369.34L172.853,369.536L173.413,369.487L173.974,369.585L174.535,366.97L175.095,335.498L175.656,368.9L176.216,369.438L176.777,369.34L177.337,369.218L177.898,369.585L178.458,368.827L179.019,368.9L179.58,369.78L180.14,369.096L180.701,369.462L181